# Inputevents Filtering
1. This file calculates number of observation per unit of measurement per itemid.
2. We iterate through all ItemID, we keep that itemID if some unit of measurement has more than 95% of all observation for that itemID, otherwise, we discard it.
3. Actually this table is quite clean, we do not have to discard any itemid. You can see the result below.
4. For some itemid that has many unit of measurement, in next step, we will adjust the measurement value based on its unit.

# Product of this File 
1. itemid of observations for inputevents.
2. unit of measurement for each itemid 

In [2]:
# DB connection:
import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
%matplotlib inline

# connect to DB

In [3]:
try:
    conn = psycopg2.connect("dbname='mimic' user='mimic' host='localhost' password='meladymimic315'")
except:
    print "I am unable to connect to the database"
print 'connection established'

connection established


# load admissionID and itemID

In [4]:
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

db = np.load('res/itemids.npy').tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

In [5]:
valid_input = []
valid_input_unit = []

# iterate thru each itemID
For each item id, we count number of observations for each unit of measurement.

For example,
IN 225883 : 98.24 : 3 : [('dose', 16477L), ('mg', 251L), ('grams', 44L)]
This means that for itemid 225883, there are:
1. 16477 records using dose as its unit of measurement.
2. 251 records using mg as its unit of measurement.
3. 44 records using grams as its unit of measurement.

dose has 98.24% over all the observations for this itemid, we can say that dose is a majority unit. 
1. We will keep this itemid because 98% is high. we can relatively safe to discard the observations that has different unit of measurement. i.e. if we discard mg and grams, we lose 251+44 records which is little, compared to 16477 records we can keep.
2. We will record main unit of measurement for this itemID as dose.

In [6]:
# inputevents
for i in input_itemid:
    cur = conn.cursor()
    cur.execute('SELECT amountuom, count(amountuom) FROM mimiciii.inputevents_mv WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+') group by amountuom')
    outputunits = cur.fetchall()
    outputunits = sorted(outputunits, key=lambda tup: tup[1])
    outputunits.reverse()
    total = 0
    for o in outputunits:
        total += o[1]
    if(total == 0 ): continue;
    percentage = float(outputunits[0][1]) / total *100.
    print "IN "+str(i) + "\t" + "{:.2f}".format(percentage) + "\t" + str(len(outputunits))+" : "+ str(outputunits)
    
    valid_input.append(i);
    valid_input_unit.append(outputunits[0][0]);

IN 225935	100.00	1 : [('mL', 4727L)]
IN 225936	100.00	1 : [('mL', 30797L)]
IN 228356	100.00	1 : [('mL', 1L)]
IN 226022	100.00	1 : [('mL', 78L)]
IN 226050	100.00	1 : [('mL', 300L)]
IN 225883	98.24	3 : [('dose', 16477L), ('mg', 251L), ('grams', 44L)]
IN 225931	100.00	1 : [('mL', 6804L)]
IN 221036	100.00	1 : [('mL', 3122L)]
IN 225851	98.44	3 : [('dose', 14371L), ('grams', 219L), ('mg', 9L)]
IN 222062	99.85	2 : [('mg', 651L), ('mcg', 1L)]
IN 222151	100.00	1 : [('mg', 51L)]
IN 225862	98.88	2 : [('dose', 265L), ('mg', 3L)]
IN 225157	100.00	1 : [('mg', 11L)]
IN 225929	100.00	1 : [('mL', 609L)]
IN 227532	100.00	1 : [('mL', 86L)]
IN 227978	100.00	1 : [('mL', 19L)]
IN 225915	100.00	1 : [('units', 5L)]
IN 225969	100.00	1 : [('grams', 1363L)]
IN 222051	100.00	1 : [('mg', 4462L)]
IN 227528	100.00	1 : [('mL', 311L)]
IN 225993	100.00	1 : [('ounces', 1L)]
IN 225830	100.00	1 : [('mL', 1373L)]
IN 226027	100.00	1 : [('mL', 8L)]
IN 226881	100.00	1 : [('mL', 26L)]
IN 225799	100.00	1 : [('mL', 96025L)]
IN 2

# save it !

In [7]:
np.save('res/filtered_input',{'id':valid_input,'unit':valid_input_unit});
print "saved !";

saved !
